In [ ]:
!pip install transformers --quiet

     |████████████████████████████████| 2.8 MB 14.5 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 895 kB 89.4 MB/s 
     |████████████████████████████████| 3.3 MB 68.9 MB/s 
     |████████████████████████████████| 636 kB 87.1 MB/s 


In [ ]:
import torch
from tqdm.notebook import tqdm
import numpy as np
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
import pandas as pd
from transformers import BertForSequenceClassification

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/PolyAI-LDN/task-specific-datasets/master/banking_data/train.csv")

In [ ]:
df.head()

,text,category
0,I am still waiting on my card?,card_arrival
1,What can I do if my card still hasn't arrived ...,card_arrival
2,I have been waiting over a week. Is the card s...,card_arrival
3,Can I track my card while it is in the process...,card_arrival
4,"How do I know if I will get my card, or if it ...",card_arrival


In [ ]:
df['category'].value_counts()

card_payment_fee_charged                            187
direct_debit_payment_not_recognised                 182
balance_not_updated_after_cheque_or_cash_deposit    181
wrong_amount_of_cash_received                       180
cash_withdrawal_charge                              177
                                                   ... 
lost_or_stolen_card                                  82
card_swallowed                                       61
card_acceptance                                      59
virtual_card_not_working                             41
contactless_not_working                              35
Name: category, Length: 77, dtype: int64

In [ ]:
possible_labels = df.category.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Refund_not_showing_up': 44,
 'activate_my_card': 71,
 'age_limit': 12,
 'apple_pay_or_google_pay': 74,
 'atm_support': 36,
 'automatic_top_up': 8,
 'balance_not_updated_after_bank_transfer': 67,
 'balance_not_updated_after_cheque_or_cash_deposit': 26,
 'beneficiary_not_allowed': 54,
 'cancel_transfer': 17,
 'card_about_to_expire': 73,
 'card_acceptance': 24,
 'card_arrival': 0,
 'card_delivery_estimate': 7,
 'card_linking': 1,
 'card_not_working': 9,
 'card_payment_fee_charged': 20,
 'card_payment_not_recognised': 27,
 'card_payment_wrong_exchange_rate': 3,
 'card_swallowed': 48,
 'cash_withdrawal_charge': 72,
 'cash_withdrawal_not_recognised': 68,
 'change_pin': 53,
 'compromised_card': 35,
 'contactless_not_working': 14,
 'country_support': 76,
 'declined_card_payment': 45,
 'declined_cash_withdrawal': 39,
 'declined_transfer': 43,
 'direct_debit_payment_not_recognised': 37,
 'disposable_card_limits': 34,
 'edit_personal_details': 28,
 'exchange_charge': 69,
 'exchange_rate': 2,
 '

In [ ]:
df['label'] = df.category.replace(label_dict)

In [ ]:
df.sample(10)

,text,category,label
5864,Why are all my card payments being declined?,declined_card_payment,45
2827,|I moved out of my old house two weeks ago and...,transfer_not_received_by_recipient,21
6779,Ive been trying to use my card for two weeks n...,reverted_card_payment?,52
2157,I need you to stop the transfer I just started...,cancel_transfer,17
7434,I tried to make a transfer like five different...,failed_transfer,57
7998,How do I use a check to top up?,top_up_by_cash_or_cheque,61
6231,I need to close my account. Your company is t...,terminate_account,47
4729,I would like to dispute a payment I didnt make,direct_debit_payment_not_recognised,37
1997,My money hasn't been deposited yet. When will ...,pending_top_up,16
9624,What do I do about my expired card?,card_about_to_expire,73


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.groupby(['category', 'label', 'data_type']).count()

text
category                                label data_type      
Refund_not_showing_up                   44    train       138
                                              val          24
activate_my_card                        71    train       135
                                              val          24
age_limit                               12    train        93
...                                                       ...
why_verify_identity                     29    val          18
wrong_amount_of_cash_received           19    train       153
                                              val          27
wrong_exchange_rate_for_cash_withdrawal 64    train       138
                                              val          25

[154 rows x 1 columns]

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train), len(dataset_val)

(8502, 1501)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [ ]:
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random
import numpy as np
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cpu


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
import os 
os.mkdir("data_volume")

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/2834 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.247385641314147
Validation loss: 0.740382984414429
F1 Score (Weighted): 0.8684196222864806


Epoch 2:   0%|          | 0/2834 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.5616407670572284
Validation loss: 0.4482262087892927
F1 Score (Weighted): 0.900418320616683


...


Epoch 3
Training loss: 0.3147051316615649
Validation loss: 0.3611338417865559
F1 Score (Weighted): 0.9125132493608572


Epoch 4:   0%|          | 0/2834 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.22106542167547583
Validation loss: 0.3418724341968636
F1 Score (Weighted): 0.9173881642861211


Epoch 5:   0%|          | 0/2834 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.20326146507542384
Validation loss: 0.3418724341968636
F1 Score (Weighted): 0.9173881642861211


In [ ]:
model.load_state_dict(torch.load('content/finetuned_BERT_epoch_5.model', map_location=torch.device('gpu')))

<All keys matched successfully>

## Model Saving

In [ ]:
import os
output_dir = "./model_save"

#create directory if not exist
if not os.path.exists(output_dir):
  os.makedirs(output_dir)
print(f"Saving model in directory  to {output_dir}")

model_to_save = model.module if hasattr(model,'module') else model
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Saving model in directory  to ./model_save


('./model_save/tokenizer_config.json',
 './model_save/special_tokens_map.json',
 './model_save/vocab.txt',
 './model_save/added_tokens.json')

In [ ]:
!zip -r "banking77_train_moddel.zip" '/content/model_save'

  adding: content/model_save/ (stored 0%)
  adding: content/model_save/tokenizer_config.json (deflated 39%)
  adding: content/model_save/special_tokens_map.json (deflated 40%)
  adding: content/model_save/pytorch_model.bin (deflated 7%)
  adding: content/model_save/config.json (deflated 72%)
  adding: content/model_save/vocab.txt (deflated 53%)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/drive/MyDrive/finetuned_BERT_epoch_5.model "/content/"

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: card_arrival
Accuracy: 22/23

Class: card_linking
Accuracy: 21/21

Class: exchange_rate
Accuracy: 16/17

Class: card_payment_wrong_exchange_rate
Accuracy: 23/25

Class: extra_charge_on_statement
Accuracy: 24/25

Class: pending_cash_withdrawal
Accuracy: 20/22

Class: fiat_currency_support
Accuracy: 19/19

Class: card_delivery_estimate
Accuracy: 14/17

Class: automatic_top_up
Accuracy: 18/19

Class: card_not_working
Accuracy: 15/17

Class: exchange_via_app
Accuracy: 16/18

Class: lost_or_stolen_card
Accuracy: 11/12

Class: age_limit
Accuracy: 17/17

Class: pin_blocked
Accuracy: 17/17

Class: contactless_not_working
Accuracy: 4/5

Class: top_up_by_bank_transfer_charge
Accuracy: 12/17

Class: pending_top_up
Accuracy: 19/22

Class: cancel_transfer
Accuracy: 24/24

Class: top_up_limits
Accuracy: 15/15

Class: wrong_amount_of_cash_received
Accuracy: 25/27

Class: card_payment_fee_charged
Accuracy: 27/28

Class: transfer_not_received_by_recipient
Accuracy: 22/26

Class: supported_cards_

## model loading

In [ ]:
# np.argmax(predictions[0])
prd =[]
for i in predictions:
  prd.append(np.argmax(i))

In [ ]:
model.load_state_dict(torch.load('/content/finetuned_BERT_epoch_5.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
from sklearn.metrics import accuracy_score
accur = accuracy_score(true_vals, prd)
print(accur)

0.9187208527648234


In [ ]:
import pandas as pd

# Load the dataset into a pandas dataframe.
val_df = pd.read_csv("https://raw.githubusercontent.com/PolyAI-LDN/task-specific-datasets/master/banking_data/test.csv")
print('Number of test sentences: {:,}\n'.format(val_df.shape[0]))
val_df['label'] = val_df.category.replace(label_dict)

Number of test sentences: 3,080



In [ ]:
val_df.head()

,text,category,label
0,How do I locate my card?,card_arrival,0
1,"I still have not received my new card, I order...",card_arrival,0
2,I ordered a card but it has not arrived. Help ...,card_arrival,0
3,Is there a way to know when my card will arrive?,card_arrival,0
4,My card has not arrived yet.,card_arrival,0


In [ ]:
# Create sentence and label lists
sentences = val_df.text.values
labels = val_df.label.values

In [ ]:
encoded_data_val = tokenizer.batch_encode_plus(
    val_df.text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(val_df.label.values)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
batch_size
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
val_loss, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
prdction =[]
for i in predictions:
  prdction.append(np.argmax(i))

In [ ]:
from sklearn.metrics import accuracy_score
accur = accuracy_score(true_vals, prdction)
print(accur)

0.9110389610389611


## Single Sentence prediction

In [ ]:
  text =['How do I locate my card?']
  encoded_data_val = tokenizer.batch_encode_plus(
      text, 
      add_special_tokens=True, 
      return_attention_mask=True, 
      pad_to_max_length=True, 
      max_length=256, 
      return_tensors='pt'
  )

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
input_ids = encoded_data_val['input_ids']
attention_mask = encoded_data_val['attention_mask']

In [ ]:
model.eval()
with torch.no_grad():
    # Forward pass, calculate logit predictions.
    result = model(input_ids, 
                    token_type_ids=None, 
                    attention_mask=attention_mask,
                    return_dict=True)
    logits = result.logits
    prediction = logits.detach().numpy()

In [ ]:
label_dict_inverse = {v: k for k, v in label_dict.items()}

In [ ]:
label_dict_inverse[np.argmax(prediction[0])]

'lost_or_stolen_card'